In [1]:
import pandas as pd
import openpyxl
import awswrangler as awr

xlsx = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas\placas_desativadas.xlsx"
df_outras = pd.read_excel(xlsx, engine = 'openpyxl')


In [ ]:
df_outras.shape[0]

140

In [2]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\sql\others_query.sql"
with open(query_path, "r") as file:
    query = file.read()

df_query = awr.athena.read_sql_query(query, database='silver')
df_query.head()  

,placa,chassi,id_placa,id_veiculo,id_carroceria,matricula,conjunto,unidade,consultor,status,...,empresa,coverage,beneficio,categoria,tipo_categoria,data_ativacao_beneficio,status_beneficio,data_atualizacao,consultor_ativo,id_beneficio
0,MKO9E68,9BM958443CB885167,244448,244448,<NA>,1223,1437,MICRO B - T.V PASQUINI SERVICOS ADMINISTRATIVO...,MF - TIAGO PASQUINI,ATIVO,...,Tag,694521,Assistência 24 Horas,PESADOS - CAVALO MECÂNICO,ATÉ 15 ANOS,2025-08-21,ATIVO,2025-08-21,S,41
1,MKO9E68,9BM958443CB885167,244448,244448,<NA>,1223,1437,MICRO B - T.V PASQUINI SERVICOS ADMINISTRATIVO...,MF - TIAGO PASQUINI,ATIVO,...,Tag,694518,Casco PT (VEÍCULO),PESADOS - CAVALO MECÂNICO,MERCEDES,2025-08-21,ATIVO,2025-08-21,S,34
2,OUU9F31,93ZC70C01E8453685,240725,240725,<NA>,1224,1438,FRANQUEADO - R M DE OLIVEIRA CORRETORES,FRANQUEADO R M DE OLIVEIRA,DESISTÊNCIA,...,Tag,694529,Assistência a Reparos (VEÍCULO),PESADOS - CAMINHÃO,PADRÃO - 4%,2025-08-20,NOVO,2025-08-22,S,37
3,MKP9E19,3GNAL7EC1BS687984,93990,93990,<NA>,1225,1439,UNIDADE GUARAPUAVA,Izidoro Svarcz,ATIVO,...,Tag,694541,Assistência a Vidros,LEVES - ECONOMICO,DE 1980 ATÉ 2019,2025-08-26,ATIVO,2025-08-26,S,42
4,MKP9E19,3GNAL7EC1BS687984,93990,93990,<NA>,1225,1439,UNIDADE GUARAPUAVA,Izidoro Svarcz,ATIVO,...,Tag,694538,RCF,LEVES - ECONOMICO,PADRÃO,2025-08-26,ATIVO,2025-08-26,S,40


In [3]:
# Vamos criar uma nova coluna em df_outras chamada 'status_beneficio_query', puxando o status correspondente da df_query pela chave 'chassi'

# Primeiro, garantir que ambas as colunas 'chassi' estejam normalizadas (string, strip, upper)
df_outras['chassi'] = df_outras['chassi'].astype(str).str.strip().str.upper()
df_query['chassi'] = df_query['chassi'].astype(str).str.strip().str.upper()

# Criar um dicionário de mapeamento chassi -> status_beneficio (pegando o primeiro status encontrado para cada chassi)
status_map = df_query.drop_duplicates('chassi').set_index('chassi')['status_beneficio'].to_dict()

# Adicionar a nova coluna em df_outras
df_outras['status_beneficio_query'] = df_outras['chassi'].map(status_map)


In [8]:
df_outras.shape[0]

140

In [ ]:
df_outras['status_beneficio_query'].value_counts()
#NOVO significa aguardando ativação

status_beneficio_query
CANCELADO      56
NOVO           52
ATIVO          20
SUBSTITUÍDO     9
RENOVAÇÃO       1
Name: count, dtype: int64

In [4]:
df_outras_naocancelados = df_outras[df_outras['status_beneficio_query'] != "CANCELADO"]
# Para ver o número de linhas:
print(df_outras_naocancelados.shape[0])
# Para ver os valores únicos de status_beneficio_query:
print(df_outras_naocancelados['status_beneficio_query'].value_counts())

84
status_beneficio_query
NOVO           52
ATIVO          20
SUBSTITUÍDO     9
RENOVAÇÃO       1
Name: count, dtype: int64


In [5]:
df_outras.shape[0]

140

In [6]:
df_outras[df_outras['status_beneficio_query']=="CANCELADO"].shape[0]

56